# Imports

In [19]:
import os
import openai

from langchain.chat_models import AzureChatOpenAI

from langchain.agents import Tool
from langchain.agents import initialize_agent
from langchain.agents import AgentType

# Setting the env variables

In [31]:
from dotenv import load_dotenv
load_dotenv()

True

In [29]:
# variables
openai.api_key = os.getenv("api_key")
openai.api_base =  os.getenv("api_base") 
openai.api_type = os.getenv("api_type")
openai.api_version = os.getenv("api_version") # this may change in the future
deployment_id=os.getenv("deployment_id_gpt_4") 

gpt_model = os.getenv("deployment_id_gpt_4") 
model_engine = os.getenv("deployment_id_gpt_4") 
embd_model = 'text-embedding-ada-002'

os.environ["OPENAI_API_TYPE"] = os.getenv("api_type")
os.environ["OPENAI_API_VERSION"] = os.getenv("api_version")
os.environ["OPENAI_API_BASE"] = os.getenv("api_base")
os.environ["OPENAI_API_KEY"] = os.getenv("api_key")

In [22]:
llm = AzureChatOpenAI(deployment_name=deployment_id,model_name=deployment_id, temperature=0.5)

# Creating langchain agent with duck-duck-go search component tool.

In [23]:
# !pip install duckduckgo-search

In [26]:
from langchain.tools import DuckDuckGoSearchRun

search = DuckDuckGoSearchRun()

# You can create the tool to pass to an agent
tools = [Tool(
    name="duckduckgo search",
    description="Use this tool to find the relavent information about the query.",
    func=search.run,
)]

search_agent = initialize_agent(
                                    tools, 
                                    llm, 
                                    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, 
                                    verbose=True, 
                                    # max_execution_time=8,
                                    max_iteration = 8,
                                    early_stopping_method="generate",
                                    handle_parsing_errors=True,
                                )

# Triggering the query, response will be coming from DuckDuckGo Search
search_agent.run(
                    "What is the hometown of the reigning men's U.S. Open champion?"
                )



> Entering new AgentExecutor chain...
I need to find out who the reigning men's U.S. Open champion is and their hometown.
Action: duckduckgo search
Action Input: reigning men's U.S. Open champion hometown
Observation: [14] In the U.S. National Championships, Richard Sears (1881-1887), William Larned (1901-1902, 1907-1911) and Bill Tilden (1920-1925, 1929) hold the record for most titles in the men's singles, with seven victories each. Rafael Nadal was the reigning champion, but chose not to participate due to both safety concerns related to the ongoing COVID-19 pandemic and the short amount of time between the US Open and the rescheduled European clay season. The US Open increased its purse to $20m for 2023, up $2.5m on 2022 and double what it was in 2016. wyndham Clark earned $3.6 million for his first Major win, up from the $3.15 million Matt Fitzpatrick received in 2022 at The Country Club in Brookline, Massachusetts. Then there are the 100 Official World Golf Ranking points (enou

"The reigning men's U.S. Open champion is Daniil Medvedev, and his hometown is Moscow, Russia."

# Creating langchain agent with bing search component tool.

In [30]:
from langchain.utilities import BingSearchAPIWrapper



os.environ["BING_SUBSCRIPTION_KEY"] = os.getenv("BING_SEARCH_V7_SUBSCRIPTION_KEY")
os.environ["BING_SEARCH_URL"] = os.getenv("BING_SEARCH_V7_ENDPOINT") 

def func_search(query_str):
    search = BingSearchAPIWrapper()
    return search.results(query_str, 1)

# You can create the tool to pass to an agent
tools = [Tool(
    name="bing search",
    description="Use this tool to find the relavent information about the query.",
    # func=search.run,
    func=func_search
)]

search_agent = initialize_agent(
                                    tools, 
                                    llm, 
                                    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, 
                                    verbose=True, 
                                    # max_execution_time=8,
                                    max_iteration = 8,
                                    early_stopping_method="generate",
                                    handle_parsing_errors=True,
                                )

# Triggering the query, response will be coming from Bing
search_agent.run(
                    "What is the hometown of the reigning men's U.S. Open champion?"
                )



> Entering new AgentExecutor chain...
I need to find out who the reigning men's U.S. Open champion is and then look up their hometown. 
Action: bing search
Action Input: reigning men's U.S. Open champion
Observation: [{'snippet': '[14] In the <b>U.S</b>. National Championships, Richard Sears (1881–1887), William Larned (1901–1902, 1907–1911) and Bill Tilden (1920–1925, 1929) hold the record for most titles in the <b>men&#39;s</b> singles, with seven victories each.', 'title': 'List of US Open men&#39;s singles champions - Wikipedia', 'link': 'https://en.wikipedia.org/wiki/List_of_US_Open_men%27s_singles_champions'}]
Thought:I need to find the most recent men's U.S. Open champion.
Action: bing search
Action Input: 2021 men's U.S. Open champion
Observation: [{'snippet': 'Daniil Medvedev defeated Novak Djokovic in the final, 6–4, 6–4, 6–4 to win the <b>men&#39;s</b> singles tennis title at the <b>2021</b> <b>US</b> <b>Open</b>. It was his first major title and he only lost one set en ro

'Moscow'

# Creating langchain agent with Google search component tool.

In [32]:
from langchain.agents import Tool
from langchain.utilities import GoogleSearchAPIWrapper

os.environ["GOOGLE_CSE_ID"] = os.getenv("GOOGLE_CSE_ID")
os.environ["GOOGLE_API_KEY"] = os.getenv("GOOGLE_API_KEY")

search = GoogleSearchAPIWrapper()

def top5_results(query):
    res = search.results(query, 2)
    print(res)
    return res

# You can create the tool to pass to an agent
tools = [Tool(
    name="google api search",
    description="Use this tool to find the relavent information about the query.",
    func=search.run,
    # func = top5_results
)]

search_agent = initialize_agent(
                                    tools, 
                                    llm, 
                                    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, 
                                    verbose=True, 
                                    # max_execution_time=8,
                                    max_iteration = 8,
                                    early_stopping_method="generate",
                                    handle_parsing_errors=True,
                                )

# Triggering the query, response will be coming from Google
search_agent.run(
                    "What is the hometown of the reigning men's U.S. Open champion?"
                )



> Entering new AgentExecutor chain...
I need to find out who the reigning men's U.S. Open champion is and their hometown.
Action: google api search
Action Input: reigning men's U.S. Open champion hometown
Observation: The US Open men's singles championship is an annual tennis tournament that is part of the US Open and was established in 1881. It is played on outdoor hard ... What is the hometown of the reigning men's U.S. Open champion? OpenAI. Params: {'model': 'text-davinci-002' ... Jul 25, 2023 ... "What is the hometown of the reigning men's U.S. Open champion?" ) Below the output from the agent, showing the follow up questions, ... List of Winners of the USA Open - Men's Singles ; 2018, Novak Đoković (Serbia), Juan Martín del Potro (Argentina) ; 2017, Rafael Nadal (Spain), Kevin Anderson ( ... Jun 15, 2020 ... World number one Novak Djokovic and reigning U.S. Open men's champion Rafa Nadal are among the players who have expressed concerns about ... Mar 31, 2018 ... That made the 

"The reigning men's U.S. Open champion is Daniil Medvedev from Russia."

# Creating langchain agent with SERP API component tool.

In [34]:
# SerpApi provides rankings and makes sense of organic search results and other search results for any domain, 
# any keyword, any country, and any language for various search engines.
from langchain import OpenAI, SerpAPIWrapper

SERPAPI_API_KEY = os.getenv("SERPAPI_API_KEY")

params = {
    "engine": "bing",
    "gl": "us",
    "hl": "en",
}
search = SerpAPIWrapper(serpapi_api_key = SERPAPI_API_KEY, params=params)

# You can create the tool to pass to an agent
tools = [Tool(
    name="Web search",
    description="Use this tool to find the relavent information about the query, the answer should be in just one word, if the answer is not found, respond with not_found.",
    func=search.run,
)]

search_agent = initialize_agent(
                                    tools, 
                                    llm, 
                                    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, 
                                    verbose=True, 
                                    # max_execution_time=8,
                                    max_iteration = 8,
                                    early_stopping_method="generate",
                                    handle_parsing_errors=True,
                                )
res = search_agent.run(
                    "What is the hometown of the reigning men's U.S. Open champion?"
                )
res



> Entering new AgentExecutor chain...
I need to find the reigning men's U.S. Open champion and their hometown.
Action: Web search
Action Input: reigning men's U.S. Open champion hometown
Observation: Monday, February 28, 2022. Daniil Medvedev achieved a major milestone at the 2021 US Open, when he won his first Grand Slam title in his third final. Fueled by the 2,000 ATP ranking points he gained from trophy run, the 26-year-old has now broken new …
Thought:I found the reigning men's U.S. Open champion, Daniil Medvedev. Now I need to find his hometown.
Action: Web search
Action Input: Daniil Medvedev hometown

ValueError: Got error from SerpAPI: Your account has run out of searches.